In [1]:
import csv
import pathlib

import networkx as nx
import numpy as np
import scipy.io

In [2]:
RESOURCES_DIR = pathlib.Path('../resources')

In [3]:
# Load graph from FiveAI paper.
with open(RESOURCES_DIR / 'hierarchy_raw/fiveai/imagenet_isa.txt') as f:
    r = csv.reader(f, delimiter=' ')
    edges = list(r)

g = nx.DiGraph()
for u, v in edges:
    g.add_edge(u, v)

In [4]:
# Load label order from ILSVRC devkit.
meta = scipy.io.loadmat(RESOURCES_DIR / 'hierarchy_raw/imagenet/meta.mat')
synsets = meta['synsets'].squeeze()
label_order = [x.item() for x in synsets[:1000]['WNID']]

In [5]:
set(label_order) == set(node for node in g if g.out_degree[node] == 0)

True

In [6]:
# TODO: Avoid duplication with prepare_hierarchy_inat.

def dfs_edges_with_order(g, order):
    visited = set()
    edges = []

    def visit(node):
        if node in visited:
            return
        visited.add(node)
        if not g.in_degree[node]:
            return
        parents = list(g.predecessors(node))
        if len(parents) > 1:
            raise ValueError('multiple parents', node, parents)
        parent, = parents
        visit(parent)
        edges.append((parent, node))
    
    for leaf in order:
        visit(leaf)

    return edges

In [7]:
edges = dfs_edges_with_order(g, label_order)

In [8]:
with open(RESOURCES_DIR / 'hierarchy/imagenet_fiveai.csv', 'w') as f:
    w = csv.writer(f)
    for edge in edges:
        w.writerow(edge)